In [7]:
import nest_asyncio
from dbgpt.agent import (
    AgentContext,
    GptsMemory,
    LLMConfig,
    ResourceLoader,
    UserProxyAgent,
)
from dbgpt.agent.expand.code_assistant_agent import CodeAssistantAgent
from dbgpt.agent.plan import AutoPlanChatManager
from dbgpt.model.proxy import OpenAILLMClient

nest_asyncio.apply()

In [ ]:
# Set your api key and api base url
# os.environ["OPENAI_API_KEY"] = "Your API"
# os.environ["OPENAI_API_BASE"] = "https://api.openai.com/v1"

In [4]:
llm_client = OpenAILLMClient(model_alias="gpt-4")
context: AgentContext = AgentContext(conv_id="test456", gpts_app_name="代码分析助手")

default_memory = GptsMemory()

resource_loader = ResourceLoader()

coder = (
    await CodeAssistantAgent()
    .bind(context)
    .bind(LLMConfig(llm_client=llm_client))
    .bind(default_memory)
    .bind(resource_loader)
    .build()
)

manager = (
    await AutoPlanChatManager()
    .bind(context)
    .bind(default_memory)
    .bind(LLMConfig(llm_client=llm_client))
    .build()
)
manager.hire([coder])

user_proxy = await UserProxyAgent().bind(context).bind(default_memory).build()


await user_proxy.initiate_chat(
    recipient=manager,
    reviewer=user_proxy,
    message="Obtain simple information about issues in the repository 'eosphoros-ai/DB-GPT' in the past three days and analyze the data. Create a Markdown table grouped by day and status.",
    # message="Find papers on gpt-4 in the past three weeks on arxiv, and organize their titles, authors, and links into a markdown table",
    # message="find papers on LLM applications from arxiv in the last month, create a markdown table of different domains.",
)